In [1]:
from keras.models import Sequential
from keras.layers import Dense,RepeatVector,Dropout
from keras.layers import Flatten
#from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
import pandas as pd
import numpy as np
import time
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler


In [2]:
data = pd.read_csv('CNA.csv')


In [4]:
data = data.filter(['close'])


In [5]:
# Lấy 100 dòng cuối cùng của df
data = data.tail(1000)
data

,close
9924,122.18
9925,121.80
9926,121.21
9927,120.96
9928,117.46
...,...
10919,191.05
10920,190.95
10921,195.40
10922,198.27


In [6]:
#Tiền xử lý
data.fillna(0, inplace=True)

# Normalize data
scaler = MinMaxScaler()
dataScale = scaler.fit_transform(data.values.reshape(-1,1))

In [7]:
#Chia dữ liệu thành train test
data = pd.DataFrame(dataScale)

#dataScale = data
train_percent = 0.7
train_size = int(len(data) * train_percent)
train_data = data.iloc[:train_size]
test_data = data.iloc[train_size:]

In [8]:
#Chuẩn bị dữ liệu để đưa vào mô hình
def prepare_data(data, size_window,size_predict,stepWindow):
    X, y = [], []
    startWindow = 0
    for i in range(len(data) - size_window - 1):
        if (len(data[(startWindow + size_window):(startWindow + size_window + size_predict) , 0]) != size_predict):
            break
        X.append(data[startWindow:(startWindow + size_window), :])
        y.append(data[(startWindow + size_window):(startWindow + size_window + size_predict) , 0])
          
       
        startWindow += stepWindow
    return np.array(X), np.array(y)


size_window = 14
size_predict = 1
stepWindow = 1
X_train, y_train = prepare_data(train_data.values, size_window, size_predict, stepWindow)
X_test, y_test = prepare_data(test_data.values, size_window, size_predict, stepWindow)

In [9]:
start_time = time.time()

In [10]:

# Tạo mô hình CNN
model = Sequential()
model.add(Conv1D(filters=32, kernel_size=6, activation='relu', input_shape=(size_window, 1)))
model.add(Conv1D(filters=32, kernel_size=6, activation='relu'))

model.add(MaxPooling1D(pool_size=2))

model.add(Flatten())
for i in range(4):
    model.add(Dense(units=100, activation='relu')) 
    model.add(Dropout(0.2))
    model.add(Dense(units=1))

# Compile mô hình với Adam optimizer và Mean Squared Error loss
model.compile(loss='mse', optimizer='adam')


In [11]:
history = model.fit(X_train, y_train, epochs=10000, batch_size=16)

Epoch 1/10000
43/43 [==============================] - 2s 2ms/step - loss: 0.1166
Epoch 2/10000
43/43 [==============================] - 0s 1ms/step - loss: 0.0249
Epoch 3/10000
43/43 [==============================] - 0s 1ms/step - loss: 0.0165
Epoch 4/10000
43/43 [==============================] - 0s 1ms/step - loss: 0.0089
Epoch 5/10000
43/43 [==============================] - 0s 1ms/step - loss: 0.0077
Epoch 6/10000
43/43 [==============================] - 0s 1ms/step - loss: 0.0074
Epoch 7/10000
43/43 [==============================] - 0s 1ms/step - loss: 0.0060
Epoch 8/10000
43/43 [==============================] - 0s 1ms/step - loss: 0.0063
Epoch 9/10000
43/43 [==============================] - 0s 1ms/step - loss: 0.0056
Epoch 10/10000
43/43 [==============================] - 0s 1ms/step - loss: 0.0056
Epoch 11/10000
43/43 [==============================] - 0s 1ms/step - loss: 0.0043
Epoch 12/10000
43/43 [==============================] - 0s 1ms/step - loss: 0.0053
Epoch 13/1000

43/43 [==============================] - 0s 1ms/step - loss: 0.0017
Epoch 197/10000
43/43 [==============================] - 0s 1ms/step - loss: 0.0017
Epoch 198/10000
43/43 [==============================] - 0s 1ms/step - loss: 0.0014
Epoch 199/10000
43/43 [==============================] - 0s 1ms/step - loss: 0.0014
Epoch 200/10000
43/43 [==============================] - 0s 1ms/step - loss: 0.0015
Epoch 201/10000
43/43 [==============================] - 0s 989us/step - loss: 0.0017
Epoch 202/10000
43/43 [==============================] - 0s 1ms/step - loss: 0.0013
Epoch 203/10000
43/43 [==============================] - 0s 989us/step - loss: 0.0014
Epoch 204/10000
43/43 [==============================] - 0s 1ms/step - loss: 0.0014
Epoch 205/10000
43/43 [==============================] - 0s 975us/step - loss: 0.0014
Epoch 206/10000
43/43 [==============================] - 0s 1ms/step - loss: 0.0015
Epoch 207/10000
43/43 [==============================] - 0s 973us/step - loss: 0.0014


43/43 [==============================] - 0s 1ms/step - loss: 0.0012
Epoch 391/10000
43/43 [==============================] - 0s 1ms/step - loss: 0.0014
Epoch 392/10000
43/43 [==============================] - 0s 1ms/step - loss: 0.0012
Epoch 393/10000
43/43 [==============================] - 0s 961us/step - loss: 0.0014
Epoch 394/10000
43/43 [==============================] - 0s 1ms/step - loss: 0.0013
Epoch 395/10000
43/43 [==============================] - 0s 1ms/step - loss: 0.0013
Epoch 396/10000
43/43 [==============================] - 0s 1ms/step - loss: 0.0014
Epoch 397/10000
43/43 [==============================] - 0s 1ms/step - loss: 0.0013
Epoch 398/10000
43/43 [==============================] - 0s 995us/step - loss: 0.0013
Epoch 399/10000
43/43 [==============================] - 0s 1ms/step - loss: 0.0013
Epoch 400/10000
43/43 [==============================] - 0s 989us/step - loss: 0.0014
Epoch 401/10000
43/43 [==============================] - 0s 1ms/step - loss: 0.0014
Ep

43/43 [==============================] - 0s 1ms/step - loss: 0.0014
Epoch 585/10000
43/43 [==============================] - 0s 1ms/step - loss: 0.0012
Epoch 586/10000
43/43 [==============================] - 0s 1ms/step - loss: 0.0010
Epoch 587/10000
43/43 [==============================] - 0s 1ms/step - loss: 0.0015
Epoch 588/10000
43/43 [==============================] - 0s 976us/step - loss: 0.0013
Epoch 589/10000
43/43 [==============================] - 0s 1ms/step - loss: 0.0011
Epoch 590/10000
43/43 [==============================] - 0s 2ms/step - loss: 0.0012
Epoch 591/10000
43/43 [==============================] - 0s 940us/step - loss: 0.0011
Epoch 592/10000
43/43 [==============================] - 0s 1ms/step - loss: 0.0012
Epoch 593/10000
43/43 [==============================] - 0s 1ms/step - loss: 0.0013
Epoch 594/10000
43/43 [==============================] - 0s 964us/step - loss: 0.0014
Epoch 595/10000
43/43 [==============================] - 0s 1ms/step - loss: 0.0012
Ep

43/43 [==============================] - 0s 1ms/step - loss: 0.0011
Epoch 779/10000
43/43 [==============================] - 0s 1ms/step - loss: 0.0012
Epoch 780/10000
43/43 [==============================] - 0s 1ms/step - loss: 0.0011
Epoch 781/10000
43/43 [==============================] - 0s 1ms/step - loss: 0.0010
Epoch 782/10000
43/43 [==============================] - 0s 1ms/step - loss: 0.0011
Epoch 783/10000
43/43 [==============================] - 0s 1ms/step - loss: 0.0011
Epoch 784/10000
43/43 [==============================] - 0s 1ms/step - loss: 0.0012
Epoch 785/10000
43/43 [==============================] - 0s 1ms/step - loss: 9.6932e-04
Epoch 786/10000
43/43 [==============================] - 0s 983us/step - loss: 0.0011
Epoch 787/10000
43/43 [==============================] - 0s 1ms/step - loss: 0.0013
Epoch 788/10000
43/43 [==============================] - 0s 1ms/step - loss: 0.0011
Epoch 789/10000
43/43 [==============================] - 0s 1ms/step - loss: 0.0010
Ep

...

43/43 [==============================] - 0s 1ms/step - loss: 9.3799e-04
Epoch 9106/10000
43/43 [==============================] - 0s 1ms/step - loss: 7.6970e-04
Epoch 9107/10000
43/43 [==============================] - 0s 1ms/step - loss: 8.4856e-04
Epoch 9108/10000
43/43 [==============================] - 0s 1ms/step - loss: 8.9052e-04
Epoch 9109/10000
43/43 [==============================] - 0s 2ms/step - loss: 7.7103e-04
Epoch 9110/10000
43/43 [==============================] - 0s 1ms/step - loss: 8.1313e-04
Epoch 9111/10000
43/43 [==============================] - 0s 1ms/step - loss: 7.9456e-04
Epoch 9112/10000
43/43 [==============================] - 0s 1ms/step - loss: 8.6309e-04
Epoch 9113/10000
43/43 [==============================] - 0s 1ms/step - loss: 8.5445e-04
Epoch 9114/10000
43/43 [==============================] - 0s 1ms/step - loss: 8.1743e-04
Epoch 9115/10000
43/43 [==============================] - 0s 1ms/step - loss: 7.9830e-04
Epoch 9116/10000
43/43 [==============

43/43 [==============================] - 0s 1ms/step - loss: 8.1358e-04
Epoch 9290/10000
43/43 [==============================] - 0s 1ms/step - loss: 7.8207e-04
Epoch 9291/10000
43/43 [==============================] - 0s 1ms/step - loss: 7.5746e-04
Epoch 9292/10000
43/43 [==============================] - 0s 1ms/step - loss: 7.8672e-04
Epoch 9293/10000
43/43 [==============================] - 0s 1ms/step - loss: 8.9107e-04
Epoch 9294/10000
43/43 [==============================] - 0s 1ms/step - loss: 9.1070e-04
Epoch 9295/10000
43/43 [==============================] - 0s 1ms/step - loss: 7.6734e-04
Epoch 9296/10000
43/43 [==============================] - 0s 1ms/step - loss: 9.4272e-04
Epoch 9297/10000
43/43 [==============================] - 0s 1ms/step - loss: 9.0923e-04
Epoch 9298/10000
43/43 [==============================] - 0s 1ms/step - loss: 7.1050e-04
Epoch 9299/10000
43/43 [==============================] - 0s 1ms/step - loss: 8.4126e-04
Epoch 9300/10000
43/43 [==============

43/43 [==============================] - 0s 1ms/step - loss: 7.8729e-04
Epoch 9474/10000
43/43 [==============================] - 0s 1ms/step - loss: 8.4020e-04
Epoch 9475/10000
43/43 [==============================] - 0s 1ms/step - loss: 7.8374e-04
Epoch 9476/10000
43/43 [==============================] - 0s 1ms/step - loss: 8.8808e-04
Epoch 9477/10000
43/43 [==============================] - 0s 1ms/step - loss: 7.5173e-04
Epoch 9478/10000
43/43 [==============================] - 0s 1ms/step - loss: 8.1705e-04
Epoch 9479/10000
43/43 [==============================] - 0s 1ms/step - loss: 8.4109e-04
Epoch 9480/10000
43/43 [==============================] - 0s 1ms/step - loss: 7.5013e-04
Epoch 9481/10000
43/43 [==============================] - 0s 1ms/step - loss: 7.8668e-04
Epoch 9482/10000
43/43 [==============================] - 0s 1ms/step - loss: 7.3306e-04
Epoch 9483/10000
43/43 [==============================] - 0s 1ms/step - loss: 6.9228e-04
Epoch 9484/10000
43/43 [==============

43/43 [==============================] - 0s 2ms/step - loss: 8.4808e-04
Epoch 9658/10000
43/43 [==============================] - 0s 1ms/step - loss: 7.7543e-04
Epoch 9659/10000
43/43 [==============================] - 0s 1ms/step - loss: 8.3596e-04
Epoch 9660/10000
43/43 [==============================] - 0s 1ms/step - loss: 8.0117e-04
Epoch 9661/10000
43/43 [==============================] - 0s 1ms/step - loss: 9.2662e-04
Epoch 9662/10000
43/43 [==============================] - 0s 1ms/step - loss: 9.4382e-04
Epoch 9663/10000
43/43 [==============================] - 0s 1ms/step - loss: 7.4457e-04
Epoch 9664/10000
43/43 [==============================] - 0s 1ms/step - loss: 7.9306e-04
Epoch 9665/10000
43/43 [==============================] - 0s 2ms/step - loss: 7.7230e-04
Epoch 9666/10000
43/43 [==============================] - 0s 1ms/step - loss: 8.7802e-04
Epoch 9667/10000
43/43 [==============================] - 0s 2ms/step - loss: 8.2953e-04
Epoch 9668/10000
43/43 [==============

43/43 [==============================] - 0s 1ms/step - loss: 7.9346e-04
Epoch 9842/10000
43/43 [==============================] - 0s 2ms/step - loss: 7.9362e-04
Epoch 9843/10000
43/43 [==============================] - 0s 1ms/step - loss: 6.9955e-04
Epoch 9844/10000
43/43 [==============================] - 0s 1ms/step - loss: 8.4027e-04
Epoch 9845/10000
43/43 [==============================] - 0s 2ms/step - loss: 7.7690e-04
Epoch 9846/10000
43/43 [==============================] - 0s 1ms/step - loss: 7.7011e-04
Epoch 9847/10000
43/43 [==============================] - 0s 1ms/step - loss: 8.9435e-04
Epoch 9848/10000
43/43 [==============================] - 0s 2ms/step - loss: 7.9166e-04
Epoch 9849/10000
43/43 [==============================] - 0s 2ms/step - loss: 7.8273e-04
Epoch 9850/10000
43/43 [==============================] - 0s 1ms/step - loss: 7.8699e-04
Epoch 9851/10000
43/43 [==============================] - 0s 2ms/step - loss: 7.5799e-04
Epoch 9852/10000
43/43 [==============

In [12]:
y_pred = model.predict(X_test)
y_pred = np.reshape(y_pred, (-1, 1))
end_time = time.time()
training_time = end_time - start_time
print(f"Training time: {training_time} seconds")

9/9 [==============================] - 0s 817us/step
Training time: 705.1755526065826 seconds


In [13]:
import h5py
from keras.models import save_model

# Lưu mô hình CNN vào file model.h5
save_model(model, 'CNA_CNN_73.h5')

# Tạo file H5 mới
with h5py.File('CNA_CNN_73_data.h5', 'w') as f:
    # Lưu giá trị y_pred
    f.create_dataset('y_test', data=y_test)
    
    # Lưu giá trị X_test
    f.create_dataset('X_test', data=X_test)
    
    # Lưu giá trị training_time
    f.create_dataset('training_time', data=training_time)
